In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
import json
import csv
from urllib.parse import urljoin
import pandas as pd

class Equitymaster(scrapy.Spider):
    name = 'equitymaster'
    allowed_domains = ['equitymaster.com']
    url = 'https://www.equitymaster.com/stock-quotes/'
    headers = {
        'Host':'www.equitymaster.com',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding':'deflate',
        'Connection': 'keep-alive',
        'Referer':'https://www.equitymaster.com/',
        'Upgrade-Insecure-Requests': 1
    }
    
    def set_custom_cookies(raw_cookie='EQNLLASTSHOWN=fivemin; _omappvp=f6shGp8hycAdEmlTH6mj39X818C6bms5jaPRkklNGVoMmVruJNib0zMOCk9mbGaMGpFwVUzQ8ZQXdQQsJ1TzHUUHHJfmNwiQ; _ga=GA1.2.1060558050.1592803218; omSeen-zao3fqjkse5xbdf22fpd=1592803220149; __atuvc=9%7C26%2C1%7C27; __atssc=google%3B1; __utma=52257654.1060558050.1592803218.1592803234.1592803234.1; __utmz=52257654.1592803234.1.1.utmcsr=menu|utmccn=(not%20set)|utmcmd=(not%20set); EQSQuotes=ALBK%5EALLAHABAD+BANK%7E; omSeen-finz4wyuz05tscppfwst=1592803268877; omSeen-dlwvieecsbccqbvg3hed=1592930470595; omSeen-fqcidkdufhoeyldom5nv=1592905522331; EQFBC=Y; ASPSESSIONIDAQRDCRTS=EELKHOCALMOCLMJDABJEDAMG; _gid=GA1.2.371805189.1593353758; _gat_UA-4311385-1=1; omSeen-b3w7iyq8sydrkjudvvcr=1593353761056; __atuvs=5ef8a621dfcf83a6000; om-b3w7iyq8sydrkjudvvcr=1593353766734'):
        cookie_list = raw_cookie.split('; ')
        cookie_dict={}
        for cookie_item in cookie_list:
            cookie_dict[cookie_item.split('=')[0]]=cookie_item.split('=')[1]
        return cookie_dict

    def start_requests(self):
        yield scrapy.Request(url=self.url, headers=self.headers, callback=self.parse_table)

    def parse_table(self, response):
        table = response.css('table.mystocks.allc')
        share_links = table.css('a')
        for link in share_links:
            link_url = link.css('a::attr(href)').extract_first()
            symbol = link_url.split('&utm_source')[0].split('=')[1]
            yield scrapy.Request(url=urljoin('https://www.equitymaster.com/stock-research/financial-data/',symbol),headers=self.headers,callback=self.parse_share)
      
    def share_parser(self, response):
        tables = response.css('table').extract()
        if len(tables)>15:
            df_table5 = pd.read_html(tables[5])[0]
            df_table6 = pd.read_html(tables[6])[0]
            df_table7 = pd.read_html(tables[7])[0]
            df_table8 = pd.read_html(tables[8])[0]
            df_table9 = pd.read_html(tables[9])[0]
            df_table10 = pd.read_html(tables[10])[0]
            df_table11 = pd.read_html(tables[11])[0]
            df_table13 = pd.read_html(tables[13])[0]
            df_table14 = pd.read_html(tables[14])[0]
            df_table15 = pd.read_html(tables[15])[0]
            return [df_table5, df_table6,df_table7, df_table8, df_table9, df_table10, df_table11, df_table13, df_table14, df_table15]
        else:
            return False
        
    def cleandata(self, response, stockname):
        try:
            df0,df1,df2,df3,df4,df5,df6,df7,df8,df9 = self.share_parser(response)            
            df0=pd.DataFrame(df0[[0,2]].values)
            df1=pd.DataFrame(df1[[0,2]].values)
            new = pd.DataFrame(['52weekHigh','52weekLow'])
            new[1] = pd.DataFrame(df1.transpose()[5].apply(lambda x:x.split('/')).values[1])
            df1 = df1.append(new)
            df3 = df3.drop(df3.columns[1], axis=1)
            df3.columns=df2.columns
            df4 = df4.drop(df4.columns[1], axis=1)
            df4.columns=df2.columns
            df5 = df5.drop(df5.columns[1], axis=1)
            df5.columns=df2.columns
            df6 = df6.drop(df6.columns[1], axis=1)
            df6.columns=df2.columns
            df8 = df8[[0,1]]
            df9 = pd.DataFrame(list(df9.transpose()[0].apply(lambda x:x.split(':')).values))
            df_details = pd.DataFrame(pd.concat([df4,df5,df6]).values)
            df_details.columns = df2.columns
            df_details.drop(df_details.columns[-1],axis=1,inplace=True)
            df_main = pd.DataFrame(pd.concat([df0,df1,df7,df8,df9]).values)
            df_melted = df_details.melt(id_vars='No. of Mths  Year Ending')
            df_melted['name']=stockname
            df_main['name']=stockname
            df_main.to_csv(f'data/main/{stockname}.csv')
            df_melted.to_csv(f'data/melted/{stockname}.csv')
        except:
            print(f"Unable to parse {stockname}")
        return
    
    def parse_share(self, response):
        stockname = response.css('span.pagesubheading strong::text').extract_first()
        self.cleandata(response, stockname)
        return
        
            
process = CrawlerProcess()
process.crawl(Equitymaster)
process.start()

2020-08-25 09:23:34 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2020-08-25 09:23:34 [scrapy.utils.log] INFO: Versions: lxml 4.5.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.2 (default, Jul 16 2020, 14:00:26) - [GCC 9.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1f  31 Mar 2020), cryptography 2.8, Platform Linux-5.4.0-42-generic-x86_64-with-glibc2.29
2020-08-25 09:23:34 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-08-25 09:23:34 [scrapy.crawler] INFO: Overridden settings:
{}
2020-08-25 09:23:34 [scrapy.extensions.telnet] INFO: Telnet Password: 3a2333a008324fed
2020-08-25 09:23:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-08-25 09:23:35 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloaderm

Unable to parse REJL


2020-08-25 09:26:01 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.equitymaster.com/stock-research/financial-data/RCFL/RASHTRIYA-CHEMICALS--FERTILIZERS-LTD-Detailed-Share-Analysis> from <GET https://www.equitymaster.com/stock-research/financial-data/RCFL>
2020-08-25 09:26:01 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.equitymaster.com/stock-research/financial-data/RBL/RBL-BANK-LTD-Detailed-Share-Analysis> from <GET https://www.equitymaster.com/stock-research/financial-data/RBL>
2020-08-25 09:26:01 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.equitymaster.com/stock-research/financial-data/RYMD/RAYMOND-LIMITED-Detailed-Share-Analysis> from <GET https://www.equitymaster.com/stock-research/financial-data/RYMD>
2020-08-25 09:26:01 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.equitymaster.com/stock-research/financial-data/RTME/RATNAMAN

Unable to parse None


2020-08-25 09:29:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.equitymaster.com/stock-research/financial-data/HCDL/HATHWAY-CABLE--DATACOM-LTD-Detailed-Share-Analysis> (referer: https://www.equitymaster.com/)
2020-08-25 09:29:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.equitymaster.com/stock-research/financial-data/GALK/GUJARAT-ALKALIES-AND-CHEMICALS-LTD-Detailed-Share-Analysis> (referer: https://www.equitymaster.com/)
2020-08-25 09:29:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.equitymaster.com/stock-research/financial-data/GJPW/GUJARAT-INDUSTRIES-POWER-COMPANY-LTD-Detailed-Share-Analysis> (referer: https://www.equitymaster.com/)
2020-08-25 09:29:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.equitymaster.com/stock-research/financial-data/GFCL/GULF-OIL-CORPORATION-LIMITED-Detailed-Share-Analysis> (referer: https://www.equitymaster.com/)
2020-08-25 09:29:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.equit

Unable to parse ALKE


2020-08-25 09:32:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.equitymaster.com/stock-research/financial-data/APLL/ALEMBIC-PHARMACEUTICALS-LTD-Detailed-Share-Analysis> (referer: https://www.equitymaster.com/)
2020-08-25 09:32:50 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.equitymaster.com/stock-research/financial-data/AJPH/AJANTA-PHARMA-LTD-Detailed-Share-Analysis> from <GET https://www.equitymaster.com/stock-research/financial-data/AJPH>
2020-08-25 09:32:50 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.equitymaster.com/stock-research/financial-data/AIAE/AIA-ENGINEERING-LIMITED-Detailed-Share-Analysis> from <GET https://www.equitymaster.com/stock-research/financial-data/AIAE>
2020-08-25 09:32:50 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.equitymaster.com/stock-research/financial-data/AHLU/AHLUWALIA-CONTRACTS-INDIA-LTD-Detailed-Share-Analysis> from <G

In [2]:
# code snippet to write to html
#         with open(f'{name}.html','w') as f:
#             f.write(response.text)
#         print(f'response saved for {name}')

In [3]:
# def set_custom_cookies(raw_cookie='EQNLLASTSHOWN=fastpro; _omappvp=f6shGp8hycAdEmlTH6mj39X818C6bms5jaPRkklNGVoMmVruJNib0zMOCk9mbGaMGpFwVUzQ8ZQXdQQsJ1TzHUUHHJfmNwiQ; _ga=GA1.2.1060558050.1592803218; omSeen-zao3fqjkse5xbdf22fpd=1592803220149; __atuvc=9%7C26%2C5%7C27; __atssc=google%3B1; __utma=52257654.1060558050.1592803218.1592803234.1592803234.1; __utmz=52257654.1592803234.1.1.utmcsr=menu|utmccn=(not%20set)|utmcmd=(not%20set); EQSQuotes=20MIC%5E20+MICRONS%7EALBK%5EALLAHABAD+BANK%7E; omSeen-finz4wyuz05tscppfwst=1592803268877; omSeen-dlwvieecsbccqbvg3hed=1592930470595; omSeen-fqcidkdufhoeyldom5nv=1592905522331; EQFBC=Y; ASPSESSIONIDAQRDCRTS=EELKHOCALMOCLMJDABJEDAMG; _gid=GA1.2.371805189.1593353758; omSeen-b3w7iyq8sydrkjudvvcr=1593353761056; om-b3w7iyq8sydrkjudvvcr=1593353766734; ASP.NET_SessionId=umeh1t55zvcxmbztemdz2m2a; __atuvs=5ef8af6b6bb012fe002; _gat_UA-4311385-1=1'):
#     cookie_list = raw_cookie.split('; ')
#     cookie_dict={}
#     for cookie_item in cookie_list:
#         cookie_dict[cookie_item.split('=')[0]]=cookie_item.split('=')[1]
#     return cookie_dict

In [5]:
# def my_parser():
#     html=''
#     with open('equitymaster.html','r') as f:
#         for line in f.read():
#             html+=line
#     response = Selector(text=html)
#     table = response.css('table.mystocks.allc')
#     company_list = table.css('a')
#     share_url = {}
#     for company in company_list[0:3]:
#         share_url[company.css('a::text').extract()[0].strip()]=company.css('a::attr(href)').extract()[0]
#     return share_url
# my_parser()

In [6]:
# def share_parser():
# #     html=''
# #     with open('ARTI.html','r') as f:
# #         for line in f.read():
# #             html+=line
        
#     response = Selector(text=html)
#     tables = response.css('table').extract()
#     df_table5 = pd.read_html(tables[5])[0]
#     df_table6 = pd.read_html(tables[6])[0]
#     df_table7 = pd.read_html(tables[7])[0]
#     df_table8 = pd.read_html(tables[8])[0]
#     df_table9 = pd.read_html(tables[9])[0]
#     df_table10 = pd.read_html(tables[10])[0]
#     df_table11 = pd.read_html(tables[11])[0]
#     df_table13 = pd.read_html(tables[13])[0]
#     df_table14 = pd.read_html(tables[14])[0]
#     df_table15 = pd.read_html(tables[15])[0]
#     return [df_table5, df_table6,df_table7, df_table8, df_table9, df_table10, df_table11, df_table13, df_table14, df_table15]

# # def table_parser(table):
# #     soup = BeautifulSoup(table, 'html.parser')
# # #     print(soup.prettify())
# #     [print(td.text) for td in soup.find_all('td')]
# #     return


# #     5,6,8,9,10,11,13,14,15 

# # share_parser()[1].transpose()

In [7]:


# string_test = '/result.asp?symbol=ZFSG&utm_source=stock-quotes&'
# len(string_test.split('&utm_source')[0].split('=')[1])

In [8]:
# import requests

# headers = {
#     'Host':'www.equitymaster.com',
#     'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0',
#     'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
#     'Accept-Language':'en-US,en;q=0.5',
#     'Accept-Encoding':'gzip, deflate, br',
#     'Connection':'keep-alive',
#     'Upgrade-Insecure-Requests':'1'
# }
# symbol='TORR'
# url = f'https://www.equitymaster.com/stock-research/financial-data/{symbol}'
# response = requests.get(url, headers=headers)
# print(response.text)


In [9]:
# def cleandata(stockname):
#     df0,df1,df2,df3,df4,df5,df6,df7,df8,df9 = share_parser()
#     df0=pd.DataFrame(df0[[0,2]].values)
#     df1=pd.DataFrame(df1[[0,2]].values)
#     new = pd.DataFrame(['52weekHigh','52weekLow'])
#     new[1] = pd.DataFrame(temp[5].apply(lambda x:x.split('/')).values[1])
#     df1 = df1.append(new)
#     df3 = df3.drop(df3.columns[1], axis=1)
#     df3.columns=df2.columns
#     df4 = df4.drop(df4.columns[1], axis=1)
#     df4.columns=df2.columns
#     df5 = df5.drop(df5.columns[1], axis=1)
#     df5.columns=df2.columns
#     df6 = df6.drop(df6.columns[1], axis=1)
#     df6.columns=df2.columns
#     df8 = df8[[0,1]]
#     df9 = pd.DataFrame(list(df9.transpose()[0].apply(lambda x:x.split(':')).values))
#     df_details = pd.DataFrame(pd.concat([df4,df5,df6]).values)
#     df_details.columns = df2.columns
#     df_details.drop(df_details.columns[-1],axis=1,inplace=True)
#     df_main = pd.DataFrame(pd.concat([df0,df1,df7,df8,df9]).values)
#     df_melted = df_details.melt(id_vars='No. of Mths  Year Ending')
#     df_melted['name']=stockname
#     df_main['name']=stockname
#     df_main.to_csv(f'data/{stockname}_main.csv')
#     df_melted.to_csv(f'data/{stockname}_melted.csv')
#     return
    

In [10]:
# cleandata(stockname='ARTI')